In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json
import glob

# 中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

ROOT = Path('..').resolve()
print(f"项目根目录: {ROOT}")

项目根目录: /home/sensen/dev/projects/-0927


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import json

# 样式设置
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('seaborn-v0_8-whitegrid')

ROOT = Path('..').resolve()
print(f"项目根目录: {ROOT}")

项目根目录: /home/sensen/dev/projects/-0927


In [ ]:
# 自动查找最新的 BT 审计结果
debug_dirs = sorted(ROOT.glob('results/debug_bt_*'))
if debug_dirs:
    RESULT_DIR = debug_dirs[-1]
    print(f"使用最新结果: {RESULT_DIR.name}")
else:
    raise FileNotFoundError("未找到 debug_bt_* 结果目录，请先运行 debug_bt_single_combo.py")

# 查找文件
equity_files = list(RESULT_DIR.glob('bt_equity_*.csv'))
trades_files = list(RESULT_DIR.glob('bt_trades_*.csv'))
summary_files = list(RESULT_DIR.glob('bt_summary_*.json'))

print(f"Equity 文件: {len(equity_files)}")
print(f"Trades 文件: {len(trades_files)}")
print(f"Summary 文件: {len(summary_files)}")

In [ ]:
# 加载数据
equity_df = pd.read_csv(equity_files[0], index_col='date', parse_dates=True)
print(f"Equity 数据: {len(equity_df)} 行")

with open(summary_files[0]) as f:
    summary = json.load(f)
    
print(f"\n策略: {summary['combo']}")
print(f"收益率: {summary['total_return_pct']}")
print(f"最大回撤: {summary['max_drawdown_pct']}")
print(f"Sharpe: {summary.get('sharpe_ratio', 'N/A')}")

In [ ]:
# 📈 资金曲线 + 回撤曲线
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True, gridspec_kw={'height_ratios': [3, 1]})

# 上图：资金曲线
ax1 = axes[0]
ax1.plot(equity_df.index, equity_df['equity'] / 1e6, 'b-', linewidth=1.5, label='Portfolio Value')
ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
ax1.fill_between(equity_df.index, 1.0, equity_df['equity'] / 1e6, 
                 where=equity_df['equity'] / 1e6 > 1.0, alpha=0.3, color='green')
ax1.fill_between(equity_df.index, 1.0, equity_df['equity'] / 1e6, 
                 where=equity_df['equity'] / 1e6 < 1.0, alpha=0.3, color='red')
ax1.set_ylabel('Portfolio Value (M)', fontsize=12)
ax1.set_title(f"BT Equity Curve\nReturn: {summary['total_return_pct']} | MaxDD: {summary['max_drawdown_pct']} | Sharpe: {summary.get('sharpe_ratio', 0):.3f}",
              fontsize=12)
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.3)

# 下图：回撤曲线
ax2 = axes[1]
ax2.fill_between(equity_df.index, equity_df['drawdown_pct'], 0, 
                 alpha=0.7, color='red', label='Drawdown')
ax2.set_ylabel('Drawdown (%)', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.set_ylim(equity_df['drawdown_pct'].min() * 1.1, 2)
ax2.legend(loc='lower left')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# 📊 月度收益热力图
monthly_returns = equity_df['equity'].resample('ME').last().pct_change(fill_method=None).dropna()

monthly_df = monthly_returns.to_frame('return')
monthly_df['year'] = monthly_df.index.year
monthly_df['month'] = monthly_df.index.month

pivot = monthly_df.pivot(index='year', columns='month', values='return')
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
pivot.columns = [month_labels[c-1] for c in pivot.columns]

fig, ax = plt.subplots(figsize=(12, 4))
im = ax.imshow(pivot.values * 100, cmap='RdYlGn', aspect='auto', vmin=-15, vmax=15)

ax.set_xticks(range(len(pivot.columns)))
ax.set_xticklabels(pivot.columns)
ax.set_yticks(range(len(pivot.index)))
ax.set_yticklabels(pivot.index)

for i in range(len(pivot.index)):
    for j in range(len(pivot.columns)):
        val = pivot.iloc[i, j]
        if not np.isnan(val):
            ax.text(j, i, f'{val*100:.1f}%', ha='center', va='center', 
                   color='black' if abs(val) < 0.1 else 'white', fontsize=8)

ax.set_title('Monthly Returns Heatmap (%)', fontsize=12)
plt.colorbar(im, ax=ax, label='Return %')
plt.tight_layout()
plt.show()

In [3]:
# 📋 交易分析
trades_df = pd.read_csv(trades_files[0])
trades_df['entry_date'] = pd.to_datetime(trades_df['entry_date'])
trades_df['exit_date'] = pd.to_datetime(trades_df['exit_date'])

print(f"总交易笔数: {len(trades_df)}")
print(f"胜率: {(trades_df['pnlcomm'] > 0).mean() * 100:.1f}%")
print(f"平均收益率: {trades_df['return_pct'].mean() * 100:.2f}%")
print(f"收益率标准差: {trades_df['return_pct'].std() * 100:.2f}%")

# 收益率分布
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 左图：收益率直方图
ax1 = axes[0]
returns_pct = trades_df['return_pct'] * 100
ax1.hist(returns_pct, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
ax1.axvline(x=0, color='red', linestyle='--', label='Zero')
ax1.axvline(x=returns_pct.mean(), color='green', linestyle='-', label=f'Mean: {returns_pct.mean():.2f}%')
ax1.set_xlabel('Return %')
ax1.set_ylabel('Count')
ax1.set_title('Trade Return Distribution')
ax1.legend()

# 右图：按标的的 PnL
ax2 = axes[1]
ticker_pnl = trades_df.groupby('ticker')['pnlcomm'].sum().sort_values(ascending=True)
colors = ['green' if v > 0 else 'red' for v in ticker_pnl]
ax2.barh(range(len(ticker_pnl)), ticker_pnl.values / 1000, color=colors, alpha=0.7)
ax2.set_yticks(range(len(ticker_pnl)))
ax2.set_yticklabels(ticker_pnl.index, fontsize=7)
ax2.axvline(x=0, color='black', linewidth=0.5)
ax2.set_xlabel('Total PnL (K)')
ax2.set_title('PnL by Ticker')

plt.tight_layout()
plt.show()

NameError: name 'trades_files' is not defined

In [ ]:
# 📊 Top/Bottom 标的分析
ticker_stats = trades_df.groupby('ticker').agg({
    'pnlcomm': ['count', 'sum', 'mean'],
    'return_pct': 'mean',
})
ticker_stats.columns = ['trades', 'total_pnl', 'avg_pnl', 'avg_return']
ticker_stats['win_rate'] = trades_df.groupby('ticker').apply(lambda x: (x['pnlcomm'] > 0).mean())
ticker_stats = ticker_stats.sort_values('total_pnl', ascending=False)

print("🏆 Top 10 盈利标的:")
print(ticker_stats.head(10).to_string())

print("\n💀 Bottom 10 亏损标的:")
print(ticker_stats.tail(10).to_string())

In [ ]:
# ✅ 年度收益统计
print("="*60)
print("📅 年度收益统计")
print("="*60)

yearly = equity_df['equity'].resample('YE').last().pct_change(fill_method=None).dropna()
for date, ret in yearly.items():
    emoji = "📈" if ret > 0 else "📉"
    print(f"{emoji} {date.year}: {ret*100:+.2f}%")

print("\n" + "="*60)
print("📋 BT 审计结论")
print("="*60)
print(f"策略: {summary['combo']}")
print(f"数据区间: 2020-01-01 ~ 2025-10-14")
print(f"初始资金: ¥{summary['start_value']:,.0f}")
print(f"最终资金: ¥{summary['end_value']:,.0f}")
print(f"")
print(f"📈 收益率: {summary['total_return_pct']}")
print(f"📉 最大回撤: {summary['max_drawdown_pct']}")
print(f"📊 Sharpe: {summary.get('sharpe_ratio', 0):.3f}")
print(f"")
print(f"🔄 总交易: {summary['total_trades']} 笔")
print(f"✅ 胜率: {summary['win_rate']*100:.1f}%")
print(f"⏱️ 平均持仓: {summary.get('avg_holding_days', 0):.1f} 天")
print(f"⚠️ Margin Failures: {summary.get('margin_failures', 0)}")
print("="*60)

In [ ]:
# 自动查找最新的 BT 审计结果
debug_dirs = sorted(ROOT.glob('results/debug_bt_*'))
if debug_dirs:
    RESULT_DIR = debug_dirs[-1]
    print(f"使用最新结果: {RESULT_DIR.name}")
else:
    raise FileNotFoundError("未找到 debug_bt_* 结果目录，请先运行 debug_bt_single_combo.py")

# 查找文件
equity_files = list(RESULT_DIR.glob('bt_equity_*.csv'))
trades_files = list(RESULT_DIR.glob('bt_trades_*.csv'))
summary_files = list(RESULT_DIR.glob('bt_summary_*.json'))

print(f"\nEquity 文件: {len(equity_files)}")
print(f"Trades 文件: {len(trades_files)}")
print(f"Summary 文件: {len(summary_files)}")

In [ ]:
# 加载数据
if equity_files:
    equity_df = pd.read_csv(equity_files[0], index_col='date', parse_dates=True)
    print(f"Equity 数据: {len(equity_df)} 行")
    print(equity_df.head())
else:
    print("⚠️ 未找到 equity 文件，请重新运行 debug_bt_single_combo.py")
    equity_df = None

if summary_files:
    with open(summary_files[0]) as f:
        summary = json.load(f)
    print(f"\n策略: {summary['combo']}")
    print(f"收益率: {summary['total_return_pct']}")
    print(f"最大回撤: {summary['max_drawdown_pct']}")
    print(f"Sharpe: {summary.get('sharpe_ratio', 'N/A')}")

In [ ]:
# 📈 资金曲线 + 回撤曲线
if equity_df is not None:
    fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
    
    # 上图：资金曲线
    ax1 = axes[0]
    ax1.plot(equity_df.index, equity_df['equity'] / 1e6, 'b-', linewidth=1.5, label='Portfolio Value')
    ax1.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
    ax1.fill_between(equity_df.index, 1.0, equity_df['equity'] / 1e6, 
                     where=equity_df['equity'] / 1e6 > 1.0, alpha=0.3, color='green')
    ax1.fill_between(equity_df.index, 1.0, equity_df['equity'] / 1e6, 
                     where=equity_df['equity'] / 1e6 < 1.0, alpha=0.3, color='red')
    ax1.set_ylabel('Portfolio Value (M)', fontsize=12)
    ax1.set_title(f"BT Equity Curve\n{summary.get('combo', 'Unknown')}\n"
                  f"Return: {summary.get('total_return_pct', 'N/A')} | MaxDD: {summary.get('max_drawdown_pct', 'N/A')} | Sharpe: {summary.get('sharpe_ratio', 0):.3f}",
                  fontsize=12)
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3)
    
    # 下图：回撤曲线
    ax2 = axes[1]
    ax2.fill_between(equity_df.index, equity_df['drawdown_pct'], 0, 
                     alpha=0.7, color='red', label='Drawdown')
    ax2.set_ylabel('Drawdown (%)', fontsize=12)
    ax2.set_xlabel('Date', fontsize=12)
    ax2.set_ylim(equity_df['drawdown_pct'].min() * 1.1, 2)
    ax2.legend(loc='lower left')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # 保存图片
    fig_path = RESULT_DIR / 'equity_drawdown.png'
    plt.savefig(fig_path, dpi=150, bbox_inches='tight')
    print(f"图片已保存: {fig_path}")
    
    plt.show()

In [ ]:
# 📊 月度收益热力图
if equity_df is not None and 'daily_return' in equity_df.columns:
    # 计算月度收益
    monthly_returns = equity_df['equity'].resample('ME').last().pct_change().dropna()
    
    # 重塑为年-月矩阵
    monthly_df = monthly_returns.to_frame('return')
    monthly_df['year'] = monthly_df.index.year
    monthly_df['month'] = monthly_df.index.month
    
    pivot = monthly_df.pivot(index='year', columns='month', values='return')
    pivot.columns = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'][:len(pivot.columns)]
    
    # 热力图
    fig, ax = plt.subplots(figsize=(12, 4))
    
    im = ax.imshow(pivot.values * 100, cmap='RdYlGn', aspect='auto', vmin=-15, vmax=15)
    
    # 标签
    ax.set_xticks(range(len(pivot.columns)))
    ax.set_xticklabels(pivot.columns)
    ax.set_yticks(range(len(pivot.index)))
    ax.set_yticklabels(pivot.index)
    
    # 在每个格子中显示数值
    for i in range(len(pivot.index)):
        for j in range(len(pivot.columns)):
            val = pivot.iloc[i, j]
            if not np.isnan(val):
                text = ax.text(j, i, f'{val*100:.1f}%', ha='center', va='center', 
                              color='black' if abs(val) < 0.1 else 'white', fontsize=8)
    
    ax.set_title('Monthly Returns Heatmap (%)', fontsize=12)
    plt.colorbar(im, ax=ax, label='Return %')
    
    plt.tight_layout()
    plt.savefig(RESULT_DIR / 'monthly_returns_heatmap.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# 📋 交易分析
if trades_files:
    trades_df = pd.read_csv(trades_files[0])
    trades_df['entry_date'] = pd.to_datetime(trades_df['entry_date'])
    trades_df['exit_date'] = pd.to_datetime(trades_df['exit_date'])
    
    print(f"总交易笔数: {len(trades_df)}")
    print(f"胜率: {(trades_df['pnlcomm'] > 0).mean() * 100:.1f}%")
    print(f"平均收益率: {trades_df['return_pct'].mean() * 100:.2f}%")
    print(f"收益率标准差: {trades_df['return_pct'].std() * 100:.2f}%")
    
    # 收益率分布
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # 左图：收益率直方图
    ax1 = axes[0]
    returns_pct = trades_df['return_pct'] * 100
    ax1.hist(returns_pct, bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    ax1.axvline(x=0, color='red', linestyle='--', label='Zero')
    ax1.axvline(x=returns_pct.mean(), color='green', linestyle='-', label=f'Mean: {returns_pct.mean():.2f}%')
    ax1.set_xlabel('Return %')
    ax1.set_ylabel('Count')
    ax1.set_title('Trade Return Distribution')
    ax1.legend()
    
    # 右图：按标的的 PnL
    ax2 = axes[1]
    ticker_pnl = trades_df.groupby('ticker')['pnlcomm'].sum().sort_values(ascending=True)
    colors = ['green' if v > 0 else 'red' for v in ticker_pnl]
    ax2.barh(range(len(ticker_pnl)), ticker_pnl.values / 1000, color=colors, alpha=0.7)
    ax2.set_yticks(range(len(ticker_pnl)))
    ax2.set_yticklabels(ticker_pnl.index, fontsize=7)
    ax2.axvline(x=0, color='black', linewidth=0.5)
    ax2.set_xlabel('Total PnL (K)')
    ax2.set_title('PnL by Ticker')
    
    plt.tight_layout()
    plt.savefig(RESULT_DIR / 'trade_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()

In [ ]:
# 📊 Top/Bottom 标的分析
if trades_files:
    # 按标的汇总
    ticker_stats = trades_df.groupby('ticker').agg({
        'pnlcomm': ['count', 'sum', 'mean'],
        'return_pct': 'mean',
    })
    ticker_stats.columns = ['trades', 'total_pnl', 'avg_pnl', 'avg_return']
    ticker_stats['win_rate'] = trades_df.groupby('ticker').apply(lambda x: (x['pnlcomm'] > 0).mean())
    ticker_stats = ticker_stats.sort_values('total_pnl', ascending=False)
    
    print("\n🏆 Top 10 盈利标的:")
    print(ticker_stats.head(10).to_string())
    
    print("\n💀 Bottom 10 亏损标的:")
    print(ticker_stats.tail(10).to_string())

In [ ]:
# ✅ 最终审计结论
if summary:
    print("="*60)
    print("📋 BT 审计结论")
    print("="*60)
    print(f"策略: {summary['combo']}")
    print(f"数据区间: 2020-01-01 ~ 2025-10-14")
    print(f"初始资金: ¥{summary['start_value']:,.0f}")
    print(f"最终资金: ¥{summary['end_value']:,.0f}")
    print(f"")
    print(f"📈 收益率: {summary['total_return_pct']}")
    print(f"📉 最大回撤: {summary['max_drawdown_pct']}")
    print(f"📊 Sharpe: {summary.get('sharpe_ratio', 0):.3f}")
    print(f"")
    print(f"🔄 总交易: {summary['total_trades']} 笔")
    print(f"✅ 胜率: {summary['win_rate']*100:.1f}%")
    print(f"⏱️ 平均持仓: {summary.get('avg_holding_days', 0):.1f} 天")
    print(f"⚠️ Margin Failures: {summary.get('margin_failures', 0)}")
    print("="*60)